In [1]:
import pandas as pd
import os
import json
import numpy as np
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import matplotsoccer as mps


In [2]:
def normalize_dataframe(df):
    json_data = json.loads(df.to_json(orient='records'))
    norm_df = pd.json_normalize(json_data)
    return norm_df



In [3]:
def rename_cols(df):
    new_cols = [i.replace('.', '_') for i in df.columns]
    df.columns = new_cols
    df = df.rename(columns={
        'period': 'period_id',
        'id':'event_id',
    })
    return df

In [16]:
data = pd.read_json("../man-city-data/StatsBomb/Data/ManCity_Arsenal_events.json")
data = rename_cols(normalize_dataframe(data))
data['time'] = data['minute'] * 60 + data['second'] - ((data['period_id'] -1) * 45  * 60 )
data 

,event_id,index,period_id,timestamp,minute,second,possession,obv_for_after,obv_for_before,obv_for_net,...,pass_inswinging,foul_committed_card_id,foul_committed_card_name,bad_behaviour_card_id,bad_behaviour_card_name,pass_miscommunication,pass_cut_back,dribble_overrun,pass_outswinging,time
0,5ab3a8e3-e6e1-4a94-b057-57b2301abac5,1,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,15347fb5-bd07-47c7-9331-b5d0d6901847,2,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7b3793be-f0aa-40b5-aedf-e2ec45e43069,3,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,b3f6b437-6189-49a7-84b3-06992c851d5d,4,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,edbc9819-3d24-4c37-adc0-06949261903b,5,1,1681776000280,0,0,2,0.015265,0.007439,0.007826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,444d8040-a64f-4d7b-a1ef-11fefd77fe15,3625,2,1681778990497,94,50,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2990
3625,e284d714-7d5f-421b-b75d-18dc784a6bcd,3626,2,1681778990497,94,50,192,0.007870,0.005570,0.002300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2990
3626,5eee8f04-dd7a-4d58-9354-7ac8bd05625b,3627,2,1681778990617,94,50,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2990
3627,eef577be-b34f-4e69-a0cd-311e7983cb6b,3628,2,1681778994982,94,54,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2994


In [6]:
data.columns

Index(['event_id', 'index', 'period_id', 'timestamp', 'minute', 'second',
       'possession', 'obv_for_after', 'obv_for_before', 'obv_for_net',
       ...
       'substitution_replacement_name', 'pass_inswinging',
       'foul_committed_card_id', 'foul_committed_card_name',
       'bad_behaviour_card_id', 'bad_behaviour_card_name',
       'pass_miscommunication', 'pass_cut_back', 'dribble_overrun',
       'pass_outswinging'],
      dtype='object', length=147)

In [9]:
import sys
 
# setting path
sys.path.append('../')
 
# # importing
from src.second_spectrum_utils import get_home_away_tracking
import src.player_velocities as vel
import src.pitch_control as pc
import src.visualizations as viz

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import pandas as pd
from kloppy import secondspectrum


In [25]:
dataset = secondspectrum.load(
    meta_data="../man-city-data/SecondSpectrum/MCI Women_s Files-001/g2312135_SecondSpectrum_meta.xml",
    raw_data="../man-city-data/SecondSpectrum/MCI Women_s Files-001/g2312135_SecondSpectrum_tracking-produced.jsonl",
    
    # Optional arguments
    sample_rate=1/1,
    coordinates="secondspectrum",
    only_alive=False)


In [26]:
df = dataset.to_df()

In [27]:
df.head()

,period_id,timestamp,frame_id,ball_state,ball_owning_team_id,ball_x,ball_y,ball_z,0746befc-a701-4b33-8faf-b3cccccf17bb_x,0746befc-a701-4b33-8faf-b3cccccf17bb_y,...,f7ae46be-3ace-423d-bee0-3482b314eb2e_d,f7ae46be-3ace-423d-bee0-3482b314eb2e_s,2c5def63-7238-4ca2-aa32-5ccbabe5a0b0_x,2c5def63-7238-4ca2-aa32-5ccbabe5a0b0_y,2c5def63-7238-4ca2-aa32-5ccbabe5a0b0_d,2c5def63-7238-4ca2-aa32-5ccbabe5a0b0_s,87dbfea7-b4c1-49ce-b210-6210f5efae55_x,87dbfea7-b4c1-49ce-b210-6210f5efae55_y,87dbfea7-b4c1-49ce-b210-6210f5efae55_d,87dbfea7-b4c1-49ce-b210-6210f5efae55_s
0,1,0.00,0,dead,home,-0.32,0.10,0.28,-11.08,1.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.04,1,alive,home,-0.82,0.22,0.28,-11.08,1.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.08,2,alive,home,-1.34,0.30,0.28,-11.07,1.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.12,3,alive,home,-1.80,0.41,0.28,-11.07,1.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0.16,4,alive,home,-2.29,0.51,0.28,-11.05,1.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df2 = df[['period_id','timestamp','frame_id','ball_state','ball_owning_team_id']]
df2

,period_id,timestamp,frame_id,ball_state,ball_owning_team_id
0,1,0.00,0,dead,home
1,1,0.04,1,alive,home
2,1,0.08,2,alive,home
3,1,0.12,3,alive,home
4,1,0.16,4,alive,home
...,...,...,...,...,...
151600,2,2997.32,151600,alive,away
151601,2,2997.36,151601,alive,away
151602,2,2997.40,151602,alive,away
151603,2,2997.44,151603,dead,away


In [30]:
# Rename columns
df2 = df2.rename(columns={'timestamp': 'time'})

# Left join the second dataframe onto the first dataframe
merged_df = pd.merge(data, df2, on=['period_id', 'time'], how='left')

# Keep only the 'frame' column
merged_df

,event_id,index,period_id,timestamp,minute,second,possession,obv_for_after,obv_for_before,obv_for_net,...,bad_behaviour_card_id,bad_behaviour_card_name,pass_miscommunication,pass_cut_back,dribble_overrun,pass_outswinging,time,frame_id,ball_state,ball_owning_team_id
0,5ab3a8e3-e6e1-4a94-b057-57b2301abac5,1,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,dead,home
1,15347fb5-bd07-47c7-9331-b5d0d6901847,2,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,dead,home
2,7b3793be-f0aa-40b5-aedf-e2ec45e43069,3,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,dead,home
3,b3f6b437-6189-49a7-84b3-06992c851d5d,4,1,1681776000000,0,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,dead,home
4,edbc9819-3d24-4c37-adc0-06949261903b,5,1,1681776000280,0,0,2,0.015265,0.007439,0.007826,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,dead,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,444d8040-a64f-4d7b-a1ef-11fefd77fe15,3625,2,1681778990497,94,50,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2990,151417,alive,away
3625,e284d714-7d5f-421b-b75d-18dc784a6bcd,3626,2,1681778990497,94,50,192,0.007870,0.005570,0.002300,...,NaN,NaN,NaN,NaN,NaN,NaN,2990,151417,alive,away
3626,5eee8f04-dd7a-4d58-9354-7ac8bd05625b,3627,2,1681778990617,94,50,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2990,151417,alive,away
3627,eef577be-b34f-4e69-a0cd-311e7983cb6b,3628,2,1681778994982,94,54,192,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2994,151517,alive,away


In [24]:
merged_df.to_csv("merged_df.csv")